# Deep Learning

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd

from gensim.models import Word2Vec

from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Dense, Masking
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras.callbacks import EarlyStopping

import joblib


Check si corre en Colab

In [3]:
try:
    DRIVE = True
    from google.colab import files
except ModuleNotFoundError:
    DRIVE = False

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
if DRIVE:
    #from google.colab import drive
    #drive.mount('/content/drive')
    #uploaded = files.upload()
    #for fn in uploaded.keys():
    #    print('User uploaded file "{name}" with length {length} bytes'.format(
    #        name=fn, length=len(uploaded[fn])))
    data = pd.read_csv('/content/drive/MyDrive/Cyberbullying_data/data.csv')
else:
    data = pd.read_csv('/content/data.csv')

In [6]:
#data = pd.read_csv('/content/data.csv')

In [6]:
data['text']=data['text'].astype(str)
data.loc[:,'target'] = pd.to_numeric(data.loc[:,'target'], downcast='integer')

data.head()

,text,target
0,this one can make an analogy in mathematical t...,0
1,clarification for you and zundarks right i ...,0
2,elected or electoral jhk,0
3,this is such a fun entry devotchka i once h...,0
4,please relate the ozone hole to increases in c...,0


In [7]:
def simple_spam(s):
    text=word_tokenize(s)
    for index in range(len(text)-1):    
        if text[index]==text[index+1]:
            text[index]=''
        s=' '.join(w for w in text if w != '')
    return s

def double_spam(s):
    text=word_tokenize(s)
    for index in range(len(text)-3):    
        if text[index]+' '+text[index+1]==text[index+2]+' '+text[index+3]:
            text[index]=''
            text[index+1]=''
        s=' '.join(w for w in text if w != '').strip()
    return s

In [8]:
from nltk.tokenize import word_tokenize

In [9]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
data['text'].apply(simple_spam)
data['text'].apply(double_spam)

0         this one can make an analogy in mathematical t...
1         clarification for you and zundarks right i sho...
2                                  elected or electoral jhk
3         this is such a fun entry devotchka i once had ...
4         please relate the ozone hole to increases in c...
                                ...                        
158878    these sources dont exactly exude a sense of im...
158879    the institute for historical review is a peerr...
158880    the way youre trying to describe it in this ar...
158881    warning there is clearly a protectionist regim...
158882    alternate option is there perhaps enough newsw...
Name: text, Length: 158883, dtype: object

In [11]:
data['target2']=['__label__'+ str(s) for s in data['target']]
data['text']= data['text'].replace('\n',' ', regex=True).replace('\t',' ', regex=True)
data.head()

,text,target,target2
0,this one can make an analogy in mathematical t...,0,__label__0
1,clarification for you and zundarks right i ...,0,__label__0
2,elected or electoral jhk,0,__label__0
3,this is such a fun entry devotchka i once h...,0,__label__0
4,please relate the ozone hole to increases in c...,0,__label__0


In [12]:
data['text2']=data['target2']+' '+data['text']
data.head()

,text,target,target2,text2
0,this one can make an analogy in mathematical t...,0,__label__0,__label__0 this one can make an analogy in mat...
1,clarification for you and zundarks right i ...,0,__label__0,__label__0 clarification for you and zundar...
2,elected or electoral jhk,0,__label__0,__label__0 elected or electoral jhk
3,this is such a fun entry devotchka i once h...,0,__label__0,__label__0 this is such a fun entry devotchk...
4,please relate the ozone hole to increases in c...,0,__label__0,__label__0 please relate the ozone hole to inc...


In [13]:
train_data2, test_data= train_test_split(data,test_size=0.3,random_state=0)


In [14]:
train_data, val_data= train_test_split(train_data2,test_size=0.15,random_state=0)


In [15]:
train_data.head()

,text,target,target2,text2
29042,you know somebodys just won it dont you m...,0,__label__0,__label__0 you know somebodys just won it d...
23247,got rid of verification tags to satisfy ...,0,__label__0,__label__0 got rid of verification tags ...
91138,i suggest you google the pictures of mehmetcik...,0,__label__0,__label__0 i suggest you google the pictures o...
21761,i changed ignore to do not agree to me thats ...,0,__label__0,__label__0 i changed ignore to do not agree t...
115617,also zapit and tv by the numbers are not ital...,0,__label__0,__label__0 also zapit and tv by the numbers a...


In [16]:
with open('/content/drive/MyDrive/Cyberbullying_data/train_fasttext.txt','w') as f:
    for datapoint in train_data['text2']:
        f.write(datapoint)
        f.write('\n')
    f.close()


In [17]:
with open('/content/drive/MyDrive/Cyberbullying_data/test_fasttext.txt','w') as f:
    for datapoint in val_data['text2']:
        f.write(datapoint)
        f.write('\n')
    f.close()

In [18]:
!pip install fasttext

     |████████████████████████████████| 68 kB 3.6 MB/s 
  Using cached pybind11-2.9.1-py2.py3-none-any.whl (211 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3134423 sha256=6b3c51e070dcf0e1f330b22c5a2c8224bc75ec3877c1a68fb8d1c62806c4a99c
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [19]:
import fasttext
model = fasttext.train_supervised('/content/drive/MyDrive/Cyberbullying_data/train_fasttext.txt',epoch=25,lr=1 ,autotuneValidationFile='/content/drive/MyDrive/Cyberbullying_data/test_fasttext.txt')
model

In [20]:
ypred=[model.predict(text) for text in test_data['text']]

In [21]:
ypred_label=[ypred[n][0][0] for n in range(len(ypred))]

In [22]:
y_pred=[int(ypred_label[n].replace('__label__','')) for n in range(len(ypred_label))]


In [23]:
y_test=test_data['target']

In [24]:

from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score

recall_score(y_test,y_pred)

0.7318982387475538

In [25]:
accuracy_score(y_test,y_pred)


0.9554390013636841

In [26]:
precision_score(y_test,y_pred)


0.7906976744186046

In [27]:
f1_score(y_test,y_pred)

0.7601626016260162

In [28]:
s = 'fuck you fuck you fuck you boobs boobs boobs boobs boobs djsaldjakl boobs boobs fuck you fuck you djskladj fuck you fuck you fuck you'
model.predict(s)

(('__label__1',), array([1.00001001]))